In [1]:
%matplotlib inline


In /home/nlplab9/anaconda3/envs/dstc_cyj/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/nlplab9/anaconda3/envs/dstc_cyj/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/nlplab9/anaconda3/envs/dstc_cyj/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/nlplab9/anaconda3/envs/dstc_cyj/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.


In [2]:
import json
import cv2
import shutil
import torch
import torchvision

import numpy as np
import torchvision.transforms as T
import seaborn as sns
import torch.nn.functional as F

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import models
from torch import nn, optim
from torch.optim import lr_scheduler
from tqdm import tqdm
from matplotlib.ticker import MaxNLocator

from matplotlib import pyplot as plt
from glob import glob
from pathlib import Path
from collections import defaultdict
from os import listdir
from os.path import isfile, join

def load_json(filename):
    with open(filename,mode="r") as f:
        return json.load(f)

def save_json(data,filename):
    with open(filename, mode="w") as f:
        json.dump(data,f)
    return

In [3]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [4]:
# remove data
# !rm -rf vision_data/color


In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [6]:
from pathlib import Path
cwd = Path().resolve()
query =cwd / "*"
print(query)
# glob(str(query))


/home/nlplab9/Desktop/youngjae/youngjae/data/*


In [7]:
# {'assetType': 'blouse_hanging',
#  'customerReview': 3.9,
#  'availableSizes': ['XS', 'S', 'XL'],
#  'color': 'red, white, yellow',
#  'pattern': 'plain',
#  'brand': 'The Vegan Baker',
#  'sleeveLength': 'long',
#  'type': 'blouse',
#  'price': 39.99,
#  'size': 'XS'}

In [8]:
scene_folder = "simmc2_scene_jsons_dstc10_public"
scenes = sorted(glob(str(scene_folder + "/*_scene.json")))
bboxes = sorted(glob(str(scene_folder + "/*_bbox.json")))


In [9]:
# for scene in scenes:
#     scene_data=load_json(join(scene_folder,scene))
#     print(scene_data.keys())
#     print(scene_data["scenes"][0].keys())
#     print(len(scene_data["scenes"][0]["objects"]))
#     print(scene_data["scenes"][0]["objects"][0].keys())
#     print(scene_data["scenes"][0]["objects"][0])
#     print(scene_data["scenes"][0]["relationships"].keys())
#     print(scene_data["scenes"][0]["relationships"])
#     break

In [10]:
image_path1=str(cwd)+  "/simmc2_scene_images_dstc10_public_part2"
image_path2=str(cwd)+ "/simmc2_scene_images_dstc10_public_part1"
image_files1 = [f for f in listdir(image_path1) if isfile(join(image_path1, f))]
image_files2 = [f for f in listdir(image_path2) if isfile(join(image_path2, f))]

In [11]:
# DATASETS = ['train', 'val', 'test']
DATASETS = ['train', 'val'] # split only by two.

In [96]:
## GENERIC VERSION
def generate_dataset(dataset_type,label_type): 
    # dataset_type : "fashion"  or "furniture" 
    # label_type : "type", "pattern", "color", "material"

    type_set = set()
    meta_file = f"{dataset_type}_prefab_metadata_all.json"
    metadata = load_json(meta_file)    
    for obj in metadata.values():
        if obj[label_type].find(",")==-1:
            type_set.add(obj[label_type])

    # build directories for data
    DATA_DIR = Path(f'vision_data/{dataset_type}/{label_type}_data')
    ORG_DATA_DIR = Path(f'vision_data/{dataset_type}/original_{label_type}_data')
    for ds in DATASETS:
        for cls in type_set:
            (DATA_DIR / ds / cls).mkdir(parents=True, exist_ok=True)

    for cls in type_set:
        (ORG_DATA_DIR / cls).mkdir(parents=True, exist_ok=True)

    # GENERIC VERSION OF GENERATING ORIGINAL DATA
    index2data={}
    index=0
    scene_folder = "simmc2_scene_jsons_dstc10_public"
    scenes = sorted(glob(str(scene_folder + "/*_scene.json")))
    for scene in scenes:
        scene_data=load_json(scene)
        scene_objects = scene_data["scenes"][0]["objects"] #object lists

        scene_image = scene.replace("_scene.json","").split("/")[1]
    #     print(image_path1 + "/" + scene_image + ".png")
    #     print(scene_image)

        if scene_image[0]=="m":
            scene_image=scene_image[2:]
        if scene_image + ".png" in image_files1:
            im = cv2.imread(image_path1 + "/" + scene_image + ".png")
        elif scene_image + ".png" in image_files2:
            im = cv2.imread(image_path2 + "/" + scene_image + ".png")
        else:
            print("there is no image available")
            continue

        # got image and scene data
        for obj in scene_objects:
            try:
                bbox = obj["bbox"]
                x=bbox[0]
                y=bbox[1]
                h=bbox[2]
                w=bbox[3]
                if h==0 or w==0:
                    raise Exception('bounding box size is zero!')
                # error if image has 
                crop_img = im[y:y+h, x:x+w]


            except Exception as e:
    #             print(e)
    #             print(x,y,h,w)
    #             print(im)
                continue

            try:
                crop_img = im[y:y+h, x:x+w]
#                 print(metadata[obj["prefab_path"]][label_type].find(","))
#                 input()
                if metadata[obj["prefab_path"]][label_type].find(",")!=-1:
#                     print(metadata[obj["prefab_path"]][label_type])
                    continue
                obj_type = metadata[obj["prefab_path"]][label_type]
                index2data[str(index)]=(str(index),obj_type,crop_img)
                index +=1

            except Exception as e:
    #             print(e)
                continue

    # save photo in the correct folder
    for key,value in index2data.items():
        cv2.imwrite(str(ORG_DATA_DIR / value[1] / (key+".jpeg")), value[2])
    
    for i, ty in enumerate(type_set):
        image_paths = np.array(glob(f'{ORG_DATA_DIR}/{ty}/*.jpeg'))
        class_name = ty
        print(f'{class_name}: {len(image_paths)}')
        np.random.shuffle(image_paths)

        ds_split = np.split(
        image_paths, 
        #     indices_or_sections=[int(.8*len(image_paths)), int(.9*len(image_paths))]
        indices_or_sections=[int(.9*len(image_paths))]

        )
        #########
        ds_split = [image_paths,ds_split[1]]
        dataset_data = zip(DATASETS, ds_split)

        for ds, images in dataset_data:
            for img_path in images:
              shutil.copy(img_path, f'{DATA_DIR}/{ds}/{class_name}/')    
    
    return DATA_DIR
        

In [115]:
DATA_DIR=generate_dataset("fashion","pattern")

design: 1510
heavy vertical stripes: 124
radiant: 589
plaid: 1094
checkered: 302
leather: 844
knit: 1098
light spots: 907
plain with stripes on side: 899
plain: 35324
cargo: 32
vertical stripes: 896
horizontal stripes: 897
twin colors: 4295
holiday: 887
star design: 0
vertical striples: 692
velvet: 637
spots: 1254
canvas: 992
streaks: 0
multicolored: 593
leapard print: 0
diamonds: 566
vertical design: 371
camouflage: 1484
leafy design: 1124
heavy stripes: 797
light stripes: 965
denim: 4117
stripes: 601
dotted: 421
light vertical stripes: 681
floral: 97
text: 703


In [116]:
# DATA_DIR = generate_dataset("furniture","materials")

In [117]:
# # For color training
# color_original_folder = "vision_data/color/color_original_data"
# ORG_DATA_DIR = Path(color_original_folder)
# color_type_set = sorted(listdir(color_original_folder))
# DATA_DIR=Path("vision_data/color/color_data")
# for ds in DATASETS:
#     for cls in color_type_set:
#         (DATA_DIR / ds / cls).mkdir(parents=True, exist_ok=True)

# for i, ty in enumerate(color_type_set):
#     image_paths = np.array(glob(f'{ORG_DATA_DIR}/{ty}/*.jpg'))
#     class_name = ty
#     print(f'{class_name}: {len(image_paths)}')
#     np.random.shuffle(image_paths)

#     ds_split = np.split(
#     image_paths, 
#     #     indices_or_sections=[int(.8*len(image_paths)), int(.9*len(image_paths))]
#     indices_or_sections=[int(.9*len(image_paths))]

#     )
#     print(ds_split)
#     input()
    

#     dataset_data = zip(DATASETS, ds_split)

#     for ds, images in dataset_data:
#         for img_path in images:
#           shutil.copy(img_path, f'{DATA_DIR}/{ds}/{class_name}/') 

In [118]:
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

transforms = {'train': T.Compose([
  T.RandomResizedCrop(size=[256,256]),
  T.RandomRotation(degrees=15),
  T.RandomHorizontalFlip(),
  T.ToTensor(),
  T.Normalize(mean_nums, std_nums)
]), 'val': T.Compose([
  T.Resize(size=[256,256]),
  T.CenterCrop(size=256),
  T.ToTensor(),
  T.Normalize(mean_nums, std_nums)
]), 'test': T.Compose([
  T.Resize(size=[256,256]),
  T.CenterCrop(size=256),
  T.ToTensor(),
  T.Normalize(mean_nums, std_nums)
]),
}

In [119]:
furniture_type_folder = Path("vision_data/furniture/type_data")
fashion_type_folder = Path("vision_data/fashion/type_data")
furniture_materials_folder = Path("vision_data/furniture/materials_data")
fashion_pattern_folder = Path("vision_data/fashion/pattern_data")
color_folder=Path("vision_data/color/color_data")
color_furn_folder=Path("vision_data/color_furn/color_data")

In [120]:
# Set input for model
# DATA_DIR=Path("vision_data/color/color_data")
# DATA_DIR=color_furn_folder
# ORG_DATA_DIR = Path("vision_data/color/original_color_data")
image_datasets = {
  d: ImageFolder(f'{DATA_DIR}/{d}', transforms[d]) for d in DATASETS
}

data_loaders = {
  d: DataLoader(image_datasets[d], batch_size=4, shuffle=True, num_workers=4) 
  for d in DATASETS
}

dataset_sizes = {d: len(image_datasets[d]) for d in DATASETS}
class_names = image_datasets['train'].classes

FileNotFoundError: Found no valid file for the classes leapard print, star design, streaks. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:
print(class_names)

In [ ]:
image_datasets["train"]
print(len(image_datasets["train"]))

In [ ]:
def imshow(inp, title=None):
  inp = inp.numpy().transpose((1, 2, 0))
  mean = np.array([mean_nums])
  std = np.array([std_nums])
  inp = std * inp + mean
  inp = np.clip(inp, 0, 1)
  plt.imshow(inp)
  if title is not None:
    plt.title(title)
  plt.axis('off')

inputs, classes = next(iter(data_loaders['train']))
# print(inputs, classes)
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
def create_model(n_classes):
  model = models.resnet34(pretrained=True)
#   model = models.resnext101_32x8d(pretrained=True)
  n_features = model.fc.in_features
  model.fc = nn.Linear(n_features, n_classes)

  return model.to(device)

In [ ]:
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  scheduler, 
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0
  
  for inputs, labels in data_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, labels)

    correct_predictions += torch.sum(preds == labels)
    losses.append(loss.item())

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  scheduler.step()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for inputs, labels in data_loader:
      inputs = inputs.to(device)
      labels = labels.to(device)

      outputs = model(inputs)

      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, labels)

      correct_predictions += torch.sum(preds == labels)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def train_model(model, data_loaders, dataset_sizes, device, n_epochs=3):
  optimizer = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)
  scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
  loss_fn = nn.CrossEntropyLoss().to(device)

  history = defaultdict(list)
  best_accuracy = 0

  for epoch in tqdm(range(n_epochs),total=n_epochs):

    print(f'Epoch {epoch + 1}/{n_epochs}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
      model,
      data_loaders['train'],    
      loss_fn, 
      optimizer, 
      device, 
      scheduler, 
      dataset_sizes['train']
    )

    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(
      model,
      data_loaders['val'],
      loss_fn,
      device,
      dataset_sizes['val']
    )

    print(f'Val   loss {val_loss} accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc.cpu())
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc.cpu())
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
      torch.save(model.state_dict(), 'best_model_state.bin')
      best_accuracy = val_acc

  print(f'Best val accuracy: {best_accuracy}')
  
  model.load_state_dict(torch.load('best_model_state.bin'))

  return model, history

In [ ]:
# Set model
base_model = create_model(len(class_names))

In [ ]:
%%time

base_model, history = train_model(base_model, data_loaders, dataset_sizes, device,n_epochs=25)

In [112]:
def plot_training_history(history):
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

  ax1.plot(history['train_loss'], label='train loss')
  ax1.plot(history['val_loss'], label='validation loss')

  ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
  ax1.set_ylim([-0.05, 1.05])
  ax1.legend()
  ax1.set_ylabel('Loss')
  ax1.set_xlabel('Epoch')

  ax2.plot(history['train_acc'], label='train accuracy')
  ax2.plot(history['val_acc'], label='validation accuracy')

  ax2.xaxis.set_major_locator(MaxNLocator(integer=True))
  ax2.set_ylim([-0.05, 1.05])
  ax2.legend()

  ax2.set_ylabel('Accuracy')
  ax2.set_xlabel('Epoch')

  fig.suptitle('Training history')

In [ ]:
plot_training_history(history)

In [ ]:
plot_training_history(history)